# Imports

In [ ]:
import sqlite3
import pandas as pd
import importlib
from datetime import datetime, timedelta
import numpy as np
import os
from pathlib import Path
import sys
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm
tqdm.pandas()  

sys.path.append(str(Path.cwd().parents[1]))

from config.constants import UKER_DIR

%load_ext autoreload
%autoreload 2

In [ ]:
def read_SQL(data_path, query, parameters):
    
    with sqlite3.connect(data_path) as conn:
        df = pd.read_sql(query, conn, params=parameters)

    return df

# Extract cohort

Extract cancer cohort

In [ ]:
query = """
SELECT DISTINCT fall.pid, fall.fid
FROM icd
JOIN fall_icd ON icd.id = fall_icd.icd
JOIN fall ON fall_icd.fid = fall.fid
WHERE icd.code LIKE 'C%'
"""
query_params = []
df =read_SQL(UKER_DIR, query, query_params)

fids_cancer = df.fid.unique().tolist()
pids_cancer = df.pid.unique().tolist()

print('# Cancer Admissions',len(fids_cancer))
print('# Cancer Patients',len(pids_cancer))

### Extract chemo cohort (ICD codes)

In [ ]:
query = """
SELECT DISTINCT fall.pid, fall.fid
FROM icd
JOIN fall_icd ON icd.id = fall_icd.icd
JOIN fall ON fall_icd.fid = fall.fid
WHERE icd.code LIKE 'Z51.1%'
   OR icd.code LIKE 'Z51.2%'
"""

query_params = []
df =read_SQL(UKER_DIR, query, query_params)

fids_chemo_icd = df.fid.unique().tolist()
pids_chemo_icd = df.pid.unique().tolist()


print('# Chemo icd  Admissions',len(fids_chemo_icd))
print('# Chemo icd Patients',len(pids_chemo_icd))

### Extract chemo cohort (procedures)

**OPS chemotherapy codes**

8-54: Cytostatic chemotherapy, immunotherapy and antiretroviral therapy

8-541 Instillation of and locoregional therapy with cytotoxic drugs and immunomodulators

**8-542 Non complex chemotherapy**

**8-543 Moderately complex and intensive block chemotherapy**

**8-544 Highly complex and intensive block chemotherapy**

8-546 Hyperthermic chemotherapy

8-547 Other immunotherapy

8-548 Highly active antiretroviral therapy (HAART)

8-549 Percutaneous isolated organ perfusion for chemotherapy

In [ ]:
query = """
SELECT DISTINCT fall.pid, fall.fid
FROM ops
JOIN fall_ops ON ops.id = fall_ops.ops
JOIN fall ON fall_ops.fid = fall.fid
WHERE ops.code LIKE '8-542%'
   OR ops.code LIKE '8-543%'
   OR ops.code LIKE '8-544%'
"""

query_params = []
df = read_SQL(UKER_DIR, query, query_params)

fids_chemo_ops = df.fid.unique().tolist()
pids_chemo_ops = df.pid.unique().tolist()


print('# Chemo ops Admissions',len(fids_chemo_ops))
print('# Chemo ops Patients',len(pids_chemo_ops))



In [ ]:
pids_chemo = set(pids_chemo_icd).union(set(pids_chemo_ops))
fids_chemo = set(fids_chemo_icd).union(set(fids_chemo_ops))

print('Total chemo admissions: ', len(fids_chemo))
print('Total chemo patients: ', len(pids_chemo))

#### Extract cancer chemo cohort

In [ ]:
cancer_chemo_patients =list( set(pids_cancer) & set(pids_chemo)) 

pids = cancer_chemo_patients

placeholders = ','.join(['?'] * len(pids)) 

query = f""" 
SELECT f.*, p.sex
FROM fall f
JOIN pat p ON f.pid = p.pid
WHERE f.pid IN ({placeholders});
"""

query_params = pids

cancer_chemo_cohort = read_SQL(UKER_DIR, query, query_params)
cancer_chemo_cohort['chemo'] = cancer_chemo_cohort['fid'].isin(fids_chemo).astype(int)
print('cancer_chemo unique admissions: ',cancer_chemo_cohort.fid.nunique())
print('cancer_chemo unique pts: ',cancer_chemo_cohort.pid.nunique())

### Clean cohort to be compatible with MIMIC-IV

In [ ]:
cancer_chemo_cohort['age'] = cancer_chemo_cohort['aufnahme_alter']
cancer_chemo_cohort['dod'] = 'NaN'


cancer_chemo_cohort['los'] = (cancer_chemo_cohort['entlassung_alter'] - cancer_chemo_cohort['aufnahme_alter'])
cancer_chemo_cohort['hospital_expire_flag'] = 0

new_columns= ['hadm_id','subject_id','admittime' ,'dischtime', 'gender',  'chemo' ,'age','dod','los','hospital_expire_flag']
cancer_chemo_cohort.columns = new_columns

Save Cancer chemo cohort

In [ ]:
new_order = ['subject_id', 'hadm_id', 'admittime', 'dischtime','los','gender', 'age', 'hospital_expire_flag', 'dod', 'chemo']
cancer_chemo_cohort = cancer_chemo_cohort[new_order]
os.makedirs('../saved_data/cohorts', exist_ok=True) 
cancer_chemo_cohort.to_csv("../saved_data/cohorts/uker_cancer_chemo_cohort.csv.gz", index=False, compression='gzip')
print("[SUCCESSFULLY SAVED COHORT DATA]")

# Extract Lab for target cohort

Load Target Cohort

In [ ]:
cancer_chemo_cohort = pd.read_csv(f'../saved_data/cohorts/uker_cancer_chemo_cohort.csv.gz', compression='gzip',  header=0)
target_cohort = cancer_chemo_cohort.copy()
pid_list = target_cohort.subject_id.unique().tolist()

Extract Lab data for the target cohort


In [ ]:

placeholders = ', '.join(['?'] * len(pid_list))


query = f"""
SELECT l.pid, f.fid, l.loinc, l.'alter', l.wert
FROM lab l
JOIN fall f ON l.pid = f.pid
JOIN pat p ON l.pid = p.pid
WHERE l.pid IN ({placeholders})
ORDER BY f.pid, l.'alter'
"""
#params =[]
params = pid_list

labs_df = read_SQL(UKER_DIR, query, params)
columns_name = ['subject_id','hadm_id','itemid','charttime','valuenum']
labs_df.columns = columns_name

print('# cc labs unique pts: ',labs_df.subject_id.nunique())
print('# cc labs unique itemids: ',labs_df.itemid.nunique())
os.makedirs('../saved_data/features', exist_ok=True) 
labs_df[['subject_id', 'hadm_id', 'charttime', 'itemid','valuenum']].to_csv(f'../saved_data/features/uker_cancer_chemo_cohort_labs.csv.gz', compression='gzip', index=False)
print("[SUCCESSFULLY SAVED LABS DATA]")


***
# Aplasia Occurrence
(only based on single ANC<0.5)

**Note:** The unit for ANC is K/uL (thousands per microliter), so a threshold of ANC < 500 cells/µL should be converted to ANC < 0.5 K/uL.

Find Aplasia cases from Lab markers (ANC < 0.5) \
In dataset max = 208.18, min = 0.0, mean = 4.33
***

Load Target cohort and corresponding Lab measurements

In [ ]:
cancer_chemo_cohort = pd.read_csv(f'../saved_data/cohorts/uker_cancer_chemo_cohort.csv.gz', compression='gzip', header=0)
cancer_chemo_labs = pd.read_csv(f'../saved_data/features/uker_cancer_chemo_cohort_labs.csv.gz', compression='gzip', header=0)

Choose target cohort and the number of days to look for aplasia occurrence

In [ ]:
target_cohort = cancer_chemo_cohort.copy()
days = 45

### Aplasia based on ANC

In [ ]:
# Extract the itemids for the given loinc code
ANC_codes = ["751-8"] # neutrophil count Loinc code

placeholders = ', '.join(['?'] * len(ANC_codes))

query = f"""
SELECT id, code
FROM loinc
WHERE code IN ({placeholders})
"""

params = ANC_codes  # pass the list directly as params

ANC_itemids = read_SQL(UKER_DIR, query, params)['id'].tolist()
print('ANC itemids are: \n ',ANC_itemids)

In [ ]:
#Filter lab measurements to include only ANC-related item IDs, and add a binary label indicating whether the ANC value is less than 0.5 or not
cancer_chemo_labs = cancer_chemo_labs.groupby(['subject_id', 'hadm_id', 'itemid', 'charttime'])["valuenum"].max().reset_index()
ANC_lab_df = cancer_chemo_labs[cancer_chemo_labs['itemid'].isin(ANC_itemids)]

mask = (ANC_lab_df['valuenum'] < 0.5)
ANC_lab_df['ANC<0.5'] = 0
ANC_lab_df.loc[mask, 'ANC<0.5'] = 1

del cancer_chemo_labs

### Aplasia based on Transfusion

In [ ]:
# Looking for OPS codes starting with 8-800

query = f"""
SELECT id
FROM ops
WHERE code LIKE ?
"""
params = ['8-800%']  # Pattern for "starts with 8-800"
transfusion_codes = read_SQL(UKER_DIR, query, params)['id'].tolist()


In [ ]:
#find all fids with the above transfusion codes
placeholders = ', '.join(['?'] * len(transfusion_codes))

query = f"""
SELECT fid
FROM fall_ops
WHERE ops IN ({placeholders})
"""
params = transfusion_codes

fid_list = read_SQL(UKER_DIR, query, params)['fid'].tolist()
print('# transfusion admissions in the entire dataset: ',len(fid_list))

common_hadm = target_cohort[target_cohort['hadm_id'].isin(fid_list)]
num_common = common_hadm['hadm_id'].nunique()
print(f"# transfusion admissions in target cohort: {num_common}")

In [ ]:
#Label target cohort admissions for transfusion encounter
target_cohort['transfusion'] = target_cohort['hadm_id'].progress_apply(lambda x: 1 if x in fid_list else 0)

Find **CURRENT** admission Aplasia occurrence based on lab measurements between admittime and dishctime

In [ ]:
def current_aplasia_occurrence(x, days, labs):
    sub_labs = labs[
        (labs["subject_id"] == x.subject_id) & 
        (labs["charttime"] >= x.admittime) & 
        (labs["charttime"] <= x.dischtime) 
    ].sort_values("charttime") 
    
    if x.transfusion == 1:
        return 1   
    if sub_labs.empty: 
        return 0
    if sub_labs["ANC<0.5"].any():
        return 1
    else:
        return 0


target_cohort["current_aplasia"] = target_cohort.progress_apply(lambda x: current_aplasia_occurrence(x, days, ANC_lab_df),axis=1)
#print(target_cohort.current_aplasia.sum())
#1244 admission before transfusion
#3599 after

Find **AFTER** admission Aplasia occurrence based on the lab measurement within certain days after discharge

In [ ]:
def after_admission_aplasia_occurrence(x,target_cohort,days,labs):
    
    #check ANC<0.5
    sub_labs = labs[
        (labs["subject_id"] == x.subject_id) & 
        (labs["charttime"]  >= x.dischtime)  & 
        (labs["charttime"]  <= x.dischtime+ days) 
        ].sort_values("charttime") 
    # check transfusions
    sub_admissions = target_cohort[
            (target_cohort["subject_id"] == x.subject_id) & 
            (target_cohort["admittime"] >= x.dischtime) & 
            (target_cohort["admittime"] <= (x.dischtime + days))
        ].sort_values("admittime") 


    
    ANC_low_rows = sub_labs[sub_labs["ANC<0.5"] == True]
    transfusion_rows = sub_admissions[sub_admissions["transfusion"] == 1]


    
    if ANC_low_rows.empty and transfusion_rows.empty:  # no aplasia 45 days after admission
        return 0, None

    else: 
        times = []
    
        if not ANC_low_rows.empty: # save first time for ANC low level
            times.append(ANC_low_rows.iloc[0]["charttime"])
        
        if not transfusion_rows.empty: # save first time for transfusion
            times.append(transfusion_rows.iloc[0]["admittime"])
        
        min_time = min(times)
        return 1, min_time # return the earliest aplasia occurrence time
        


target_cohort[["next_aplasia", "next_aplasia_time"]]  = target_cohort.progress_apply(lambda x: pd.Series(after_admission_aplasia_occurrence(x, target_cohort,days, ANC_lab_df)), axis=1)
target_cohort["next_aplasia"] = target_cohort["next_aplasia"].astype(int)



Split Negative and Positive cases for target cohort

In [ ]:
def split_aplasia_cases(x, days,target_cohort):
    
    if x.chemo == 0 or (x.chemo == 1 and x.current_aplasia==1): # if there is no chemo or if both chemo and aplasia are present the admission is not considered!
        return 0
    if x.chemo ==1 and x.current_aplasia == 0 and x.hospital_expire_flag ==0:
        sub = target_cohort[
            (target_cohort["subject_id"] == x.subject_id) & 
            (target_cohort["hadm_id"]  != x.hadm_id) &  # avoid selecting the same admission as next where the los for admission is 0
            (target_cohort["admittime"]  >= x.dischtime) &
            #(target_cohort["admittime"] <= (x.dischtime + timedelta(days=days)))
            (target_cohort["admittime"] <= (x.dischtime + days))
        ].sort_values("admittime")  

        # remove this for UKEr dataset we do not have info regarding death
        '''#remove admissions where patient died within 30 days of discharge
        if sub.empty and x.dod <= (x.dischtime + timedelta(days=days)): 
        #if sub.empty and x.dod <= (x.dischtime + days):
            return 0'''
        
        if sub.empty:  # no readmission
            if (x["next_aplasia"] == 1): return 2
            if (x["next_aplasia"] == 0): return 1
            
        if not sub.empty: # if there is readmission
            
            if (sub["chemo"] == 0).all(): # if no chemo in readmissions
                if (x["next_aplasia"] == 1): return 2
                if (x["next_aplasia"] == 0): return 1
                
            if (sub["chemo"] == 1).any(): # if chemo in readmissions
                first_chemo_time = sub.loc[sub["chemo"] == 1, "admittime"].min()
               
                if (x["next_aplasia"] == 0):  # no aplasia in 45 days 
                    return 1
                if (x["next_aplasia"] == 1): # if aplasia in 45 days
                    if x.next_aplasia_time < first_chemo_time:  # if aplasia occurs before next chemo the admission is positive else not considered
                        return 2
                    else: 
                        return 0
                
target_cohort_temp = target_cohort[target_cohort.subject_id == 14663]
target_cohort["aplasia_case"] = target_cohort.progress_apply(lambda x: split_aplasia_cases(x, days,target_cohort), axis=1)


In [ ]:
pos_case = target_cohort[target_cohort["aplasia_case"] == 2]
neg_case= target_cohort[target_cohort["aplasia_case"] == 1]
none_case= target_cohort[target_cohort["aplasia_case"] == 0]


print('-----------------------------------')
print('positive admissions',pos_case.hadm_id.nunique())
print('-----------------------------------')
print('negative admissions',neg_case.hadm_id.nunique())
print('-----------------------------------')
print('not determined admissions',none_case.hadm_id.nunique())
print('-----------------------------------')
print('Positive percentage: ',pos_case.hadm_id.nunique()/(pos_case.hadm_id.nunique() + neg_case.hadm_id.nunique())*100) 

Save Cohort

In [ ]:
pos_case.loc[:, 'label'] = np.ones(pos_case.shape[0]).astype(int)
neg_case.loc[:, 'label'] = np.zeros(neg_case.shape[0]).astype(int)

cohort_output="uker_cohort" + "_" + "aplasia" + "_" + str(days) + "_days"
cohort = pd.concat([pos_case, neg_case], axis=0)

cohort = cohort.drop(columns =['hospital_expire_flag','chemo','current_aplasia','next_aplasia','next_aplasia_time','aplasia_case','transfusion'])

cohort.to_csv("../saved_data/cohorts/"+cohort_output+".csv.gz", index=False, compression='gzip')
print("[ COHORT SUCCESSFULLY SAVED ]")

print(cohort_output)